# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial uses [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [45]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [46]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [47]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [48]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [49]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [50]:
messages = [
    {"role": "system", "content": "use tone as a high school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. This city is not only the political hub but also home to many important national landmarks and institutions. If you have any more questions about U.S. geography or history, feel free to ask!


Add assistant messages to teach LLM what `##` is.

In [51]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [52]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

### Connect to the MongoDB cluster

In [53]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

### Extract Tweets

In [54]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [55]:
tweet_data = []
for tweet in result:
    tweet_data.append(tweet['tweet']['text'])

print(tweet_data)

["Couldn't you quote your idol Mao, or would you fumble the election? https://t.co/W6fuRj0b2k https://t.co/UBzwK0mMUq", '@EndWokeness So now we can expect 13,001 votes from dead people in this election?', 'RT @NickJFuentes: Trump 2024:\n• Mass Immigration (legally)\n• Trans Kids (with parental consent)\n• No New Wars (except with Iran)\n• Free Spe…', 'Californians have until 10/21 to register to vote online for the November 5 election. Check your voter status or register at: https://t.co/DuBK9nYqI3 https://t.co/lgCri2T6W5', 'RT @AccountableGOP: Republican Former Georgia Lt. Gov. @GeoffDuncanGA on Donald Trump: “He’s a liar by nature…because he didn’t win and bec…', "RT @Starboy2079: Why BJP doesn't understand a simple fact that as number of Bangladeshi Muslims and Rohingyas will keep increasing, their p…", 'RT @MTGrepp: BREAKING 🚨 Donald Trump just EXPOSED the Justice Department for Election interference 🔥 \n\n"Virginia cleaned up its Voter Rolls…', '@LauraLoomer Their time will be com

In [56]:
print('Number of tweets: ',len(tweet_data))

Number of tweets:  200


### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [57]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets cover a wide range of topics related to upcoming elections, primarily focusing on the U.S. presidential election. Key themes include accusations of election interference, voter fraud, and media bias. There are discussions about voter registration deadlines, particularly in California, and the importance of voter turnout. Some tweets express skepticism about the integrity of the election process, referencing past elections and potential fraud. Others highlight political strategies and campaign activities, including criticisms of candidates like Donald Trump and Kamala Harris. Additionally, there are mentions of international election-related issues, such as alleged foreign interference and the influence of dark money in politics. Overall, the tweets reflect a highly charged and polarized political atmosphere leading up to the election.


In [36]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss election-related topics, including voter registration deadlines, accusations of election interference, and predictions about Trump's chances in the upcoming election.


In [37]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people discuss AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets primarily focus on election-related discussions, with minimal mention of AI. The conversation revolves around election integrity, voter registration, and political strategies. There is a strong emphasis on the upcoming election's significance, with various opinions on candidates and potential outcomes.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [38]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [39]:
for tweet in tweet_data:
    flag_help(tweet)

===
harassment
RT @Starboy2079: Why BJP doesn't understand a simple fact that as number of Bangladeshi Muslims and Rohingyas will keep increasing, their p…
===
harassment
@LauraLoomer Their time will be coming to an end soon once Trump wins this election. They will be running and hiding. No more protection from this treasonous administration.
===
harassment
RT @maddenifico: Some of you self-righteous motherfuckers on the left are once again overthinking this. It's as if you faux-prog frauds lea…
===
harassment
@BretBaier This will be the biggest pillow fight of the election process. She was indeed the Border Czar, she lied to us about the health of President Biden, used tax payer $ to pay for a convicted felons sex change,  no wars when Trump was President and what is her position on the freedom… https://t.co/PEgwpFAM5q
===
harassment
The media and the left spend all year claiming crime is down. They “fact check” Trump at the debate on it. Weeks before the election, the FBI quietly “up

### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [40]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

你不能引用你的偶像毛泽东的话吗，还是你会搞砸选举？https://t.co/W6fuRj0b2k https://t.co/UBzwK0mMUq
@EndWokeness 所以现在我们可以期待在这次选举中有13,001张来自已故者的选票吗？
转发 @NickJFuentes: 特朗普2024： • 大规模移民（合法） • 变性儿童（需父母同意） • 不发动新战争（除了与伊朗） • 免费…
加利福尼亚州居民可以在10月21日之前在线注册，以参加11月5日的选举。请在此查看您的选民状态或注册：https://t.co/DuBK9nYqI3 https://t.co/lgCri2T6W5
RT @AccountableGOP: 乔治亚州前共和党副州长@GeoffDuncanGA 谈到唐纳德·特朗普：“他天生就是个骗子……因为他没有赢得胜利，因为…”
转发 @Starboy2079: 为什么印度人民党（BJP）不明白一个简单的事实，即随着孟加拉国穆斯林和罗兴亚人的数量不断增加，他们的政…


KeyboardInterrupt: 

In [41]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

Oh, do tell, couldn't you muster a quote from your dear idol Mao, or would that be a bit too much for your little election charade? https://t.co/W6fuRj0b2k https://t.co/UBzwK0mMUq
@EndWokeness Oh, splendid! Are we to anticipate a delightful encore of 13,001 votes from the dearly departed in this election? How utterly predictable.
Ah, yes, the grand plan for 2024, courtesy of the Donald. A delightful concoction of mass immigration, but only if it's legal, mind you. Trans kids, but only if mummy and daddy say it's alright. No new wars, except, of course, a little tiff with Iran. And let's not forget the promise of free... well, something or other. How positively riveting!


KeyboardInterrupt: 

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [42]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a single word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "negative",
  "emotion": "sarcasm",
  "mentioned": [],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "skepticism",
  "mentioned": ["@EndWokeness"],
  "support": "Republican"
}
{
  "sentiment": "neutral",
  "emotion": "indifference",
  "mentioned": ["NickJFuentes", "Trump"],
  "support": "Republican"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": [],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "disapproval",
  "mentioned": [
    "GeoffDuncanGA",
    "Donald Trump"
  ],
  "support": "Democratic"
}
{
  "sentiment": "negative",
  "emotion": "frustration",
  "mentioned": ["BJP"],
  "support": "Republican"
}
{
  "sentiment": "negative",
  "emotion": "outrage",
  "mentioned": ["Donald Trump"],
  "support": "Republican"
}
{
  "sentiment": "negative",
  "emotion": "anger",
  "mentioned": ["LauraLoomer", "Trump"],
  "support": "Republican"
}
{
  "sentiment": "negative",
  "emotion": "frustration",
  "mentioned

KeyboardInterrupt: 

In [43]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "Election Fraud Allegations",
    "Donald Trump's 2024 Campaign",
    "Voting and Voter Registration",
    "Media and Election Coverage",
    "Political Figures and Statements",
    "Election Integrity and Security",
    "Polling and Election Predictions",
    "Election Laws and Legal Challenges",
    "Political Campaigns and Strategies",
    "Public Sentiment and Voter Engagement"
  ]
}


### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [58]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 200/200 [03:22<00:00,  1.01s/it]


In [59]:
print(analysis_result)

['{\n  "sentiment": "negative",\n  "emotion": "sarcasm",\n  "mentioned": [],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "skepticism",\n  "mentioned": ["@EndWokeness"],\n  "support": "Republican"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "skepticism",\n  "mentioned": ["Nick J Fuentes", "Trump"],\n  "support": "Republican"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": [],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "disapproval",\n  "mentioned": [\n    "GeoffDuncanGA",\n    "Donald Trump"\n  ],\n  "support": "Democratic"\n}', '{\n  "sentiment": "negative",\n  "emotion": "frustration",\n  "mentioned": [\n    "Starboy2079"\n  ],\n  "support": "Republican"\n}', '{\n  "sentiment": "negative",\n  "emotion": "anger",\n  "mentioned": ["Donald Trump"],\n  "support": "Republican"\n}', '{\n  "sentiment": "negative",\n  "emotion": "anger",\n  "mentioned": ["LauraLoomer", "Trump"],\n  "support": "Re

In [60]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democratic and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democratic count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "Democratic count": 23,
  "Republican count": 66,
  "people name": {
    "Democratic": {
      "common sentiments": ["negative", "neutral", "positive"],
      "common emotions": ["frustration", "concern", "supportive", "admiration", "excitement", "satisfaction", "disapproval"]
    },
    "Republican": {
      "common sentiments": ["negative", "neutral", "positive"],
      "common emotions": ["frustration", "anger", "concern", "skepticism", "anticipation", "informative", "outrage"]
    }
  }
}


## Create a chatbot

In [61]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chabot answer user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people name in the {delimiter}{analysis_summary}{delimiter} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  what are the tweets about


Chatbot: The tweets are primarily about the upcoming elections, political figures, and related topics. There are discussions on voting processes, election interference, opinions on different political leaders, sentiments about the Democratic and Republican parties, and potential outcomes of the elections. Additionally, there are mentions of specific states and voting regulations, media coverage of the elections, and opinions on policies and campaign strategies. Various opinions and emotions are expressed in relation to the current political climate and electoral prospects.


## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
